In [26]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [27]:
# Get directory of sound files
dirloc = os.path.join(os.getcwd(), "speech-emotion-recognition-ravdess-data")

In [34]:
# Extract features from sound file - mfcc, chroma, mel
def extract_features(file, mfcc, chroma, mel):
    with soundfile.SoundFile(file) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [64]:
import speech_recognition as sr

In [65]:
# initialize speech recognizer
r = sr.Recognizer()

In [67]:
# Extract text from sound file
def extract_text(file):
    with sr.AudioFile(file) as source:
        # load audio
        audio_data = r.record(source)
        # convert speech to text
        text = r.recognize_google(audio_data)
    return text

In [35]:
# RAVDESS emotions
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

# Observed emotions
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [101]:
# Create function to load data and extract features for each sound file
def load_data(directory):
    x = []
    y = []
    for file in glob.glob(f"{directory}/Actor_*/*.wav"):
        filename=os.path.basename(file)
        emotion=emotions[filename.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_features(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return x, y

In [71]:
# Test one file to extract text
test_filename = dirloc + "/Actor_01/03-01-01-01-01-01-01.wav"

In [73]:
test_text = extract_text(test_filename)

In [74]:
print(test_text)

kids are talking by the door


In [87]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

In [100]:
sia = SIA()
pol_scores = sia.polarity_scores(test_text)
for i in sorted(pol_scores):
    print('{0}: {1}, '.format(i, pol_scores[i]), end='')

compound: 0.0, neg: 0.0, neu: 1.0, pos: 0.0, 

In [119]:
# Create function to extract text from each sound file and detect sentiment
def get_text(directory):
    xt = []
    yt = []
    for file in glob.glob(f"{directory}/Actor_*/*.wav"):
        try:
            #filename=os.path.basename(file)
            text = extract_text(file)
            sia = SIA()
            pol_score = sia.polarity_scores(text)
            sentiment = 0
            if pol_score["compound"] > 0.2:
                sentiment = 1
            elif pol_score["compound"] < -0.2:
                sentiment = -1
            else:
                sentiment = 0
            xt.append(text)
            yt.append(sentiment)
        except:
            print("Error")
    return xt, yt

In [ ]:
xt, yt = get_text(dirloc)

Error
Error
Error
Error
Error
Error
Error
Error
Error
Error


In [114]:
# Load all data and extract features
x, y = load_data(dirloc)

[-5.37004395e+02  3.16663990e+01 -6.54209137e+00  4.56760597e+00
 -7.38780451e+00 -1.36847029e+01 -1.78605824e+01 -1.18284941e+01
 -2.25547388e-01 -1.18684692e+01 -8.85833073e+00  2.02371097e+00
 -9.43830681e+00 -4.10037231e+00 -6.45495605e+00 -3.87169814e+00
 -4.01648903e+00 -4.76854229e+00 -5.11756539e-01  1.33889759e+00
  2.94026017e+00  3.80391932e+00  2.59524417e+00  5.09492064e+00
 -7.14929760e-01  2.14967346e+00 -1.82484186e+00  7.07248390e-01
 -4.69545901e-01 -3.38963093e-03 -5.91821969e-01 -1.61439348e-02
  1.01429716e-01  8.82703483e-01  2.87238121e-01  3.08487654e-01
 -8.90636146e-01 -1.47073340e+00 -7.07513213e-01 -3.91681612e-01
  4.82439458e-01  4.45954591e-01  4.22002167e-01  3.90913755e-01
  3.83992523e-01  4.37711895e-01  4.52970594e-01  4.81909454e-01
  5.30166030e-01  5.87507606e-01  5.77323794e-01  5.37041843e-01
  2.69676730e-05  3.71418246e-05  4.45121150e-05  6.87977517e-05
  7.41148615e-05  1.02672179e-03  8.21552984e-03  5.98948188e-02
  1.94896534e-01  2.97292

[-6.35356628e+02  5.09899445e+01  4.74262714e+00  1.16014023e+01
 -7.12209368e+00 -1.33440351e+01 -7.26556873e+00 -4.23065138e+00
 -5.78687143e+00 -8.53597927e+00 -3.87439203e+00 -3.00005841e+00
 -3.80521441e+00 -3.04585505e+00 -7.49980307e+00 -2.32797694e+00
 -7.63275433e+00 -2.80068254e+00 -5.20480156e+00 -4.11755848e+00
 -5.01925564e+00 -2.45621729e+00 -2.88215804e+00  9.59040880e-01
 -1.83878705e-01  2.37249225e-01 -1.18874311e+00 -2.32521698e-01
  1.09656382e+00  2.76318789e+00 -9.21254531e-02 -1.51361421e-01
 -1.09584622e-01  1.10133052e+00  2.04021168e+00  3.32617021e+00
  8.90217602e-01  2.85069913e-01 -9.26226377e-01  7.41408587e-01
  5.24960160e-01  4.82828200e-01  4.33898538e-01  4.60107386e-01
  4.48855370e-01  4.56398934e-01  4.66484159e-01  4.64744955e-01
  4.69187617e-01  5.35345078e-01  4.96889263e-01  4.84215409e-01
  3.44806767e-06  1.46504162e-05  6.30551804e-05  6.06026151e-05
  7.04450213e-05  1.10593569e-02  2.76770871e-02  1.80515125e-02
  8.27841386e-02  6.44988

[-5.72798767e+02  3.39717560e+01 -8.65464866e-01  9.83142376e+00
 -6.53630161e+00 -1.26989756e+01 -1.59462070e+01 -5.93936634e+00
 -5.45345306e-01 -9.45522022e+00 -4.94131088e+00  2.52729368e+00
 -7.09275293e+00 -3.02548790e+00 -8.28550339e+00 -1.01638973e+00
 -6.98715210e+00 -4.67347431e+00 -4.11958122e+00 -3.25644517e+00
  1.06724346e+00 -1.21178412e+00  1.68110645e+00  1.22406375e+00
  1.93857050e+00  1.98845363e+00  1.42902124e+00  2.38663006e+00
 -1.42643332e-01  3.33820432e-01  1.21099734e+00  6.03510737e-01
  3.95047575e-01 -1.42649233e+00 -2.96071559e-01  1.95743227e+00
  1.75266337e+00 -1.91680813e+00  2.40967572e-01 -4.41396274e-02
  6.22584879e-01  5.51106811e-01  5.27676284e-01  5.19621909e-01
  5.06062210e-01  5.46696603e-01  6.07905865e-01  6.16744459e-01
  7.25386083e-01  7.50479877e-01  7.57150590e-01  7.50533819e-01
  2.75677685e-05  4.54727087e-05  8.65611873e-05  7.28117739e-05
  3.42946470e-04  4.12285794e-03  2.37874165e-02  5.01674674e-02
  3.21704835e-01  3.26539

[-5.11530518e+02  4.19838066e+01  3.34410834e+00  3.15248847e+00
 -2.23368216e+00 -1.44155920e+00 -5.87005615e+00 -4.01316166e+00
 -1.02179289e+01  2.04074550e-02 -3.00158501e+00 -1.95047319e+00
 -4.85164833e+00  1.62309182e+00 -4.54272461e+00 -2.55141377e+00
  5.83554566e-01 -2.42890143e+00 -4.48838949e+00 -1.33316982e+00
 -2.80177021e+00 -2.16536593e+00 -2.11251283e+00 -8.16686392e-01
 -3.50523257e+00 -1.64626396e+00 -1.61089325e+00 -3.40492070e-01
  2.47892427e+00  3.05097771e+00  4.10478878e+00  3.99516964e+00
  4.32052183e+00  4.93820763e+00  3.26600409e+00  3.40001035e+00
  3.55625725e+00  3.26788020e+00  3.20615077e+00  2.52272439e+00
  6.05281413e-01  6.17592216e-01  6.45809174e-01  6.62251711e-01
  6.91520631e-01  7.12320089e-01  6.68972611e-01  6.89375401e-01
  7.01206744e-01  6.91676199e-01  6.97686195e-01  7.01914608e-01
  2.21301033e-03  5.07417251e-04  2.94553989e-04  1.52793888e-03
  1.42821055e-02  1.37351766e-01  6.59262598e-01  7.87627876e-01
  5.01318991e-01  1.41495

[-4.79099457e+02  4.73573875e+01  3.05132389e+00  5.15209341e+00
 -1.21261334e+00  1.16754568e+00 -2.06599641e+00 -5.87107420e+00
 -9.30265427e+00  3.62328482e+00 -2.06894445e+00  4.20952439e-01
 -3.33839512e+00  1.45120656e+00 -1.61266315e+00 -1.52767181e-01
 -7.83829927e-01 -1.94963086e+00 -3.91514182e+00  1.24321902e+00
 -1.55234087e+00 -4.21785384e-01  1.41583234e-01  1.98609865e+00
  2.79398990e+00  5.43795967e+00  2.98527026e+00  5.10846138e+00
  3.31731892e+00  4.25332594e+00  3.74734235e+00  5.58031607e+00
  5.92765856e+00  8.10410690e+00  5.78255129e+00  2.29244542e+00
  2.38348007e+00  2.12954211e+00  2.79195666e+00  2.51537514e+00
  5.51745653e-01  6.13249838e-01  6.15310669e-01  6.51798904e-01
  6.78617835e-01  6.97158873e-01  6.48319662e-01  6.30599678e-01
  6.12135172e-01  6.49296939e-01  6.75572872e-01  5.90812385e-01
  1.14740229e+00  1.15550622e-01  1.43808192e-02  1.64090854e-03
  8.76754615e-03  1.10160530e-01  3.31499845e-01  7.19524980e-01
  1.01937163e+00  2.11353

...

[-4.24215210e+02  2.73477192e+01 -4.91085863e+00  7.59495497e+00
  6.33820295e-01  3.63384724e+00 -2.08779073e+00 -5.59186554e+00
  3.67226303e-01  2.55897593e+00 -4.68294239e+00  5.31894541e+00
 -2.63833928e+00  4.13596344e+00  6.62980080e-01 -1.19527531e+00
  1.55546665e+00 -2.26842260e+00  1.45276725e-01  2.73681593e+00
  5.40393686e+00  7.64786577e+00  6.58598280e+00  7.63920498e+00
  4.06828213e+00  5.22334766e+00  5.21971703e+00  4.44161844e+00
  4.73789787e+00  6.00250721e+00  6.36253500e+00  6.26108646e+00
  2.46566796e+00  3.22243333e+00  1.85688734e+00  4.17521524e+00
  3.64859843e+00  3.46392560e+00  3.54961228e+00  3.55836916e+00
  6.64387286e-01  6.55091465e-01  7.41852283e-01  7.70406306e-01
  7.49568939e-01  7.15515494e-01  7.28390634e-01  8.05264115e-01
  7.96159685e-01  7.97766447e-01  7.83727765e-01  7.07873166e-01
  8.18412685e+00  2.04969183e-01  1.71486512e-02  4.39079432e-03
  8.45943205e-03  8.11372995e-02  6.42200649e-01  7.02635646e-01
  1.14172769e+00  1.18341

[-4.47449768e+02  2.93737164e+01 -7.06178761e+00  5.11886978e+00
  2.30054879e+00  1.00610542e+00 -4.09336948e+00 -6.68092251e+00
 -1.13481474e+00  3.56505156e+00 -5.19753313e+00  3.93090177e+00
 -3.45782518e+00  1.56151915e+00 -7.09743917e-01 -3.46678543e+00
  2.35261869e+00  1.33665979e+00  4.02203608e+00  4.93059301e+00
  5.25934505e+00  4.51688242e+00  5.05381536e+00  6.90610075e+00
  6.90618134e+00  7.29588032e+00  4.98375559e+00  6.13688183e+00
  4.33720922e+00  6.57101393e+00  4.31124210e+00  2.80841351e+00
  2.12335348e+00  4.55904436e+00  3.61745119e+00  3.68084192e+00
  2.38161302e+00  5.56434822e+00  4.80410337e+00  3.52772450e+00
  6.67706490e-01  6.47014499e-01  6.55983150e-01  6.90200686e-01
  7.91227162e-01  7.69048750e-01  7.34998941e-01  7.97888398e-01
  7.77422369e-01  7.59623051e-01  7.72394359e-01  7.06711948e-01
  8.71609986e-01  3.78487958e-03  9.55217169e-04  2.09030905e-03
  7.16381893e-03  6.79257661e-02  1.26077712e-01  4.51029271e-01
  1.47951210e+00  3.25686

[-5.62147949e+02  4.42137222e+01  4.35887289e+00  1.47478456e+01
  5.67066193e+00  5.97509241e+00 -1.41344583e+00 -6.79561090e+00
  4.15932989e+00 -1.16596437e+00 -2.65885854e+00  4.88085604e+00
 -1.59394288e+00  7.51522839e-01  2.98244208e-01 -3.66539448e-01
  4.81883436e-01  1.18518806e+00 -2.12114143e+00 -8.37730408e-01
 -1.94590294e+00 -1.36669517e+00  3.19660306e-02  7.71772563e-01
 -2.03341961e-01  7.13449061e-01 -7.32906938e-01 -9.69960332e-01
 -4.64346617e-01 -3.87242250e-02 -4.73943084e-01 -1.27242303e+00
 -1.05530620e+00 -1.35010123e+00 -3.01721573e+00 -1.64397275e+00
 -1.37196839e+00  1.12615252e+00  1.44096458e+00  2.06304526e+00
  6.46440506e-01  6.40386462e-01  6.13760352e-01  6.18851721e-01
  6.48045719e-01  6.73266709e-01  7.19980240e-01  7.98972666e-01
  7.97150433e-01  7.92630374e-01  8.20357442e-01  7.61651874e-01
  4.97456291e-04  2.61368143e-04  1.20401022e-03  1.89166665e-02
  1.08840950e-01  1.83789864e-01  2.32423306e-01  2.04909481e-02
  3.39682214e-02  9.68846

[-4.37169891e+02  1.13870735e+01 -1.83064709e+01 -2.75053954e+00
 -1.34189663e+01 -1.82613766e+00 -1.39737997e+01 -1.26981106e+01
 -7.87977123e+00 -4.73424196e+00 -1.46971762e+00  1.03251505e+01
  1.54752598e+01  5.72566128e+00 -2.31912240e-01 -1.56723773e+00
  2.18077469e+00  3.01222873e+00 -1.74391532e+00 -1.24808037e+00
 -2.50434804e+00  2.50129193e-01  1.12852252e+00  2.11667991e+00
 -1.26316857e+00  1.82107925e+00 -3.12491894e+00 -1.56449604e+00
 -9.57636759e-02  5.79273105e-01 -4.25723135e-01  1.23300529e+00
  4.07378733e-01  6.43189073e-01  3.26393533e+00  1.74929929e+00
  3.92862409e-02  1.09240437e+00 -8.72538984e-01 -6.86574638e-01
  5.87888360e-01  5.51521301e-01  4.79578495e-01  4.71416295e-01
  4.79777336e-01  4.83184755e-01  5.40689230e-01  6.01774037e-01
  6.15685225e-01  5.67306280e-01  5.54769158e-01  5.54033637e-01
  5.62287169e-03  5.04457997e-03  1.46860455e-03  6.22576918e-04
  4.89954720e-04  1.72081066e-03  2.18222453e-03  1.23723678e-03
  6.63956220e-04  4.96887

KeyboardInterrupt: 

In [49]:
X = np.array(x)

In [55]:
#Split the dataset into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [56]:
#for subdir in os.listdir(dirloc):
    #subdir_folder = os.path.join(dirloc,subdir)
   # if os.path.isdir(subdir_folder):
       # for file in os.listdir(subdir_folder):
           # filename = os.path.join(subdir_folder, file)
            #extract_features(filename)

In [57]:
print(X_train.shape[0], X_test.shape[0])

614 154


In [58]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [60]:
# Initialize MLPClassifier instance from Sci-Kit Learn - multi-layer perceptron classifier utilizes internal neural network for classification
classifier = MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [61]:
# Fit MLPClassifier model with training data
classifier.fit(X_train, y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [62]:
# Predict values based on test set
y_pred = classifier.predict(X_test)

In [63]:
# Create confusion matrix and calculate accuracy score
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[34  3  1  0]
 [ 0 35  3  1]
 [ 1  5 21  5]
 [ 2 11 10 22]]


0.7272727272727273